In [68]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, date

## CFM 101: Group Assignment - Python Roboadvisor
### Team Number: 15
### Team Member Names: Landon Trinh, Ethan Zemelman, Jessie Deng
### Team Strategy Chosen: SAFE

## Goal
- Our team has decided to target dynamically building the safest portfolio
- Given a file of unknown stock tickers, our roboadvisor will run from November 25, 2023 to December 4, 2023
- The deviation will be calculated by taking calculating the difference between the final value of the portfolio and the initial portfolio value ($750,000)
- Ultimately, our aim is to have our portfolio deviate in nominal value as close to zero


## Introduction
> "Theory will only take you so far." - J. Robert Oppenheimer

In theory, our trading strategy should produce a portfolio that deviates in nominal value the least by taking into consideration the following:
- Beta
- Diversification
- Correlation
- Standard deviation
- Expected returns

Our goal will be to tell a convincing story to WHY we are picking our stocks. We will calculate and discuss statistics, display and intepret graphs, and explain our thought process

## 1. Setup
Before implementing our trading strategy, we will initialize required and useful constants as part of the rules:
- Currency of valid stocks (USD or CAD)
- Required average monthly volume (150,00 shares)
- The number of stocks we wish to purchase on the start date (10-22 stocks)
- Time interval (Janurary 1, 2023 - October 31, 2023)
- Minimum number of trading days for month (18 days)
- Minimum stock weighting: (100/2n)%, n = number of stocks in portfolio
- Maximum stock weighting (20%)
- Initial investment amount ($750,000 CAD)
- Buying date of roboadvisor (November 25, 2023 - December 4, 2023)
- Trading fee for each stock trade ($4.95 CAD)

In the end, our roboadvisor should create two DataFrames:
1. "Portfolio_Final"
- Index: Starts at 1 and ends at number of stocks in portfolio
- Headings: Ticker, Price (price of stock on Nov 25), Currency (CAD or USD), Shares, Value, Weight (adds to 100%)

2. "Stocks_Final"
We should output this DataFrame to a CSV file titled "Stocks_Group_15.csv"
- Index: Same as "Portfolio Final"
- Headings: Tickers and Shares from "Portfolio_Final"


In [69]:
# Investment amount (CAD)
capital = 750000

# Number of stocks to buy for portfolio
num_stocks = 15

# Maximum and minimum weightings of each stock in portfolio
min_weight = 1 / (2 * num_stocks)
max_weight = 0.20

# Start and end date for roboadvisor
# start_date = "2023-11-25"
# end_date = "2023-12-04"

# Filtering requirements
valid_currency = ["CAD", "USD"]
min_trading_days = 18
required_avg_volume = 150000

## 2. Filtering
After reading in the CSV file containg stock tickers, we must filter the list of stocks to make sure they are valid stock tickers according to the following rules:
- Include stocks that have an average monthly volume of at leaest 150,000 shares based on Jan 1, 2023 - Oct 31, 2023 (drop any months that don't have at least 18 trading days)
- Stock denominated in USD or CAD

In [70]:
# Read in CSV ticker file
tickers = pd.read_csv("tickers_example.csv", header=None)
tickers = tickers.rename(columns={0: "ticker"})
tickers_lst = tickers["ticker"].tolist()
tickers_lst

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AGN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTN',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

In [71]:
# Set parameters
filter_start_date = "2023-01-01" # Jan 1, 2023
filter_end_date = "2023-10-31" # Oct 31, 2023
filter_interval = "1mo"

In [72]:
# Keep stocks with average monthly volume of 150k shares - drop months with less than 18 trading days
def valid_volume(stock_ticker):

    monthly_volumes = []

    for month in range(1,10):
        # Set monthly date intervals
        month_start_date = str(date(2023, month, 1))
        month_end_date = str(date(2023, month + 1, 1))

        # Retrieve volume data for month
        monthly_volume_hist = stock_ticker.history(interval="1d", start=month_start_date, end=month_end_date).Volume
        monthly_volume_hist.index = monthly_volume_hist.index.strftime("%Y-%m-%d")

        # Retrive number of trading days and averge monthly volume
        trading_days = len(monthly_volume_hist)
        monthly_volume = monthly_volume_hist.sum()

        # Add monthly volume to list if valid number of trading days
        if trading_days >= min_trading_days:
            monthly_volumes.append(monthly_volume)

    # Determine whether average monthly volume meets requirement
    avg_monthly_volume = sum(monthly_volumes) / len(monthly_volumes)
    if avg_monthly_volume < required_avg_volume:
        return False
    return True

## 3. Stock Analysis
- Get standard deviation of each stock
- Get the beta of each stock
- Get the expected returns of each stock
- Create a correlation matrix of all the stocks

## 4. Portfolio Optimization
- Create random weights for each stock, and create n number of random portfolios
- Choose the portfolio with the lowest expected returns

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.